In [2]:
import pandas as pd

# Load the data from the provided CSV files
bills_df = pd.read_csv('csv/bills.csv')
documents_df = pd.read_csv('csv/documents.csv')
history_df = pd.read_csv('csv/history.csv')
people_df = pd.read_csv('csv/people.csv')
rollcalls_df = pd.read_csv('csv/rollcalls.csv')
sponsors_df = pd.read_csv('csv/sponsors.csv')
votes_df = pd.read_csv('csv/votes.csv')

bills_df.head()


17016

In [106]:
pip install seaborn openai pandas plotly

8220.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 294 kB 165 kB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 321 kB/s eta 0:00:01
     |████████████████████████████████| 66 kB 570 kB/s eta 0:00:011
     |████████████████████████████████| 103 kB 330 kB/s eta 0:00:01
     |████████████████████████████████| 244 kB 374 kB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 727 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Define keywords to search for in titles and descriptions
keywords = ["israel", "palestine", "netanyahu", 'palestinian', 'israeli']

# Function to check if any of the keywords are in a given text
def contains_keywords(text, keywords):
    text = text.lower() if isinstance(text, str) else ""
    return any(keyword in text for keyword in keywords)

# Filter the bills that mention any of the keywords in their title or description
relevant_bills = bills_df[
    bills_df['title'].apply(lambda x: contains_keywords(x, keywords)) |
    bills_df['description'].apply(lambda x: contains_keywords(x, keywords))
]


# Show the relevant bills
relevant_bills.head()

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link
330,1657930,2041,HB340,2,Engrossed,2023-11-02,Hamas and Other Palestinian Terrorist Groups I...,To impose sanctions with respect to foreign su...,0,NaN,2023-11-06,Read the second time. Placed on Senate Legisla...,https://legiscan.com/US/bill/HB340/2023,https://www.congress.gov/bill/118th-congress/h...
542,1683909,2041,HB552,1,Introduced,2023-01-26,No Tax Dollars for the United Nation’s Immigra...,To prohibit United States contributions to the...,2305,House Foreign Affairs,2023-01-26,Referred to the House Committee on Foreign Aff...,https://legiscan.com/US/bill/HB552/2023,https://www.congress.gov/bill/118th-congress/h...
977,1704525,2041,HB987,1,Introduced,2023-02-10,Prime Minister Golda Meir Commemorative Coin Act,To require the Secretary of the Treasury to mi...,2304,House Financial Services,2023-02-10,Referred to the House Committee on Financial S...,https://legiscan.com/US/bill/HB987/2023,https://www.congress.gov/bill/118th-congress/h...
1092,1714878,2041,HB1102,1,Introduced,2023-02-17,UNRWA Accountability and Transparency Act,To withhold United States contributions to the...,2305,House Foreign Affairs,2023-02-17,Referred to the House Committee on Foreign Aff...,https://legiscan.com/US/bill/HB1102/2023,https://www.congress.gov/bill/118th-congress/h...
1208,1722213,2041,HB1218,1,Introduced,2023-02-27,Expediting Israeli Aerial Refueling Act of 2023,To require the Secretary of the Air Force to p...,2299,House Armed Services,2023-02-27,Referred to the House Committee on Armed Servi...,https://legiscan.com/US/bill/HB1218/2023,https://www.congress.gov/bill/118th-congress/h...


In [ ]:
from openai import OpenAI


# Using OpenAI API in order to rate all the bills based on how Pro Israel their Title and Description is


titles_Description_Bills = []
##Create the array that will be pushed into ChatGPT
for _, bill in relevant_bills.iterrows():
    individual_bill = []
    if bill['bill_number'] == bill['title']:
            individual_bill.append(bill['bill_number'])
            individual_bill.append(bill['title'])
    else:
        individual_bill.append(bill['bill_number'])
        individual_bill.append(bill['title'])
        individual_bill.append(bill['description'])
    titles_Description_Bills.append(individual_bill)

StringTitles = str(titles_Description_Bills)
client = OpenAI(api_key="OpenAI_API_KEY")
##Prompt Given to ChatGPT
prompt = """
Below is a list of all the recent bills that include Israel, I will include the number, title, description.
I want to rank them based on how positive they are towards Israel. 
I want you to rank them and give them a score 1-5. 1 being very anti Israel, and 5 being very pro Israel. 
If Israel is not the center of the bill, prefer to rank it lower.  
I want you to return these in a form of an array [number, score]. 
I want you to be very accurate and delicate. 
If the bill mentions having a ceasefire made, consider it on the more anti Israeli Side
""" + StringTitles

response = client.chat.completions.create(
  model = 'gpt-4o-mini',
  messages = [{"role": 'user', 'content' : prompt}],
  temperature=0.3,
  )

print(response.choices[0].message.content)

In [ ]:
##Returned Array from ChatGPT
scores = [
    ['SR784', 5],
    ['HB9206', 5],
    ['HB9160', 5],
    ['HB9153', 5],
    ['SB4779', 5],
    ['SB4770', 5],
    ['HB9115', 4],
    ['HR1383', 5],
    ['HB8445', 4],
    ['HB8345', 4],
    ['HR1323', 5],
    ['HB8232', 5],
    ['SR758', 5],
    ['SB4045', 3],
    ['SB4551', 4],
    ['SB4537', 4],
    ['HB8629', 3],
    ['HB8537', 3],
    ['SB4410', 5],
    ['SB4408', 5],
    ['HB8437', 5],
    ['HB8369', 5],
    ['HB8299', 4],
    ['SB4337', 5],
    ['HR1227', 4],
    ['HR1231', 1],
    ['HR1219', 5],
    ['HR1224', 5],
    ['SB4311', 4],
    ['SR682', 5],
    ['SR680', 5],
    ['SB4285', 5],
    ['HB8295', 5],
    ['HR1211', 5],
    ['SB3874', 5],
    ['SR505', 5],
    ['HB8254', 4],
    ['HB8034', 5],
    ['HR1160', 4],
    ['HR1143', 5],
    ['HR883', 5],
    ['HR1149', 5],
    ['HR1147', 5],
    ['HR1137', 4],
    ['HR1117', 4],
    ['HR1125', 4],
    ['HB7866', 5],
    ['HR1107', 3],
    ['SR620', 5],
    ['SR617', 5],
    ['HR1092', 5],
    ['HCR99', 3],
    ['SR589', 3],
    ['HB7552', 4],
    ['HR1041', 5],
    ['HB7493', 4],
    ['HB7372', 4],
    ['HR966', 5],
    ['HB7271', 5],
    ['HB7217', 5],
    ['HB6679', 5],
    ['HB7202', 5],
    ['SB3707', 4],
    ['HR980', 4],
    ['HB7111', 5],
    ['HB7113', 5],
    ['SR504', 1],
    ['HB6367', 3],
    ['HR942', 3],
    ['SR497', 5],
    ['HR930', 5],
    ['HB6694', 5],
    ['SR445', 5],
    ['HB6553', 5],
    ['HR902', 1],
    ['HB6511', 4],
    ['HR793', 5],
    ['HR888', 5],
    ['HR895', 1],
    ['SB3343', 5],
    ['HR889', 5],
    ['HR887', 4],
    ['HJR102', 1],
    ['HB6126', 5],
    ['SB3299', 5],
    ['SR462', 5],
    ['SR408', 5],
    ['HR849', 5],
    ['HR845', 5],
    ['SB3238', 4],
    ['HB340', 5],
    ['HB3266', 4],
    ['SB3210', 5],
    ['HB6211', 5],
    ['HB6200', 5],
    ['HB6196', 5],
    ['HR838', 4],
    ['SB3183', 5],
    ['HR834', 5],
    ['SB3168', 5],
    ['HB6118', 5],
    ['HB6113', 5],
    ['HB6099', 5],
    ['SB3135', 5],
    ['HB6060', 5],
    ['HR803', 5],
    ['HR797', 4],
    ['HR796', 5],
    ['HR789', 5],
    ['HR771', 5],
    ['HR775', 5],
    ['HB5986', 5],
    ['HB5959', 5],
    ['HR786', 1],
    ['HB5932', 5],
    ['HR778', 4],
    ['HR768', 5],
    ['HR770', 5],
    ['HB5952', 5],
    ['SR417', 5],
    ['SB3083', 5],
    ['SR413', 5],
    ['SB3081', 5],
    ['SB3052', 5],
    ['SB2956', 4],
    ['SCR14', 5],
    ['HCR61', 5],
    ['HCR57', 5],
    ['SR307', 5],
    ['HB4709', 5],
    ['HB4564', 5],
    ['SB2240', 4],
    ['HB4076', 4],
    ['HB3907', 4],
    ['SB1802', 4],
    ['HB3792', 4],
    ['HR409', 5],
    ['HB3393', 5],
    ['SB1637', 5],
    ['SB1647', 5],
    ['HR388', 1],
    ['SB1504', 5],
    ['HB3103', 1],
    ['SR188', 5],
    ['HR346', 5],
    ['SB1300', 5],
    ['HR311', 5],
    ['HB2531', 5],
    ['SB1143', 4],
    ['HB2454', 4],
    ['HB2322', 3],
    ['HB1777', 4],
    ['HB1270', 3],
    ['HB1218', 5],
    ['HB1102', 5],
    ['SB510', 5],
    ['SB431', 5],
    ['HB987', 5],
    ['HR92', 5],
    ['HB552', 4]
]


##Adding the score into the Relevant Bills DF
for score in scores:
    relevant_bills.loc[relevant_bills['bill_number'] == score[0], "Score"] = score[1]

relevant_bills.head()

In [20]:
relevant_bills['Score'].unique()

relevant_bills.head()

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link,Score
330,1657930,2041,HB340,2,Engrossed,2023-11-02,Hamas and Other Palestinian Terrorist Groups I...,To impose sanctions with respect to foreign su...,0,NaN,2023-11-06,Read the second time. Placed on Senate Legisla...,https://legiscan.com/US/bill/HB340/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
542,1683909,2041,HB552,1,Introduced,2023-01-26,No Tax Dollars for the United Nation’s Immigra...,To prohibit United States contributions to the...,2305,House Foreign Affairs,2023-01-26,Referred to the House Committee on Foreign Aff...,https://legiscan.com/US/bill/HB552/2023,https://www.congress.gov/bill/118th-congress/h...,4.0
977,1704525,2041,HB987,1,Introduced,2023-02-10,Prime Minister Golda Meir Commemorative Coin Act,To require the Secretary of the Treasury to mi...,2304,House Financial Services,2023-02-10,Referred to the House Committee on Financial S...,https://legiscan.com/US/bill/HB987/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
1092,1714878,2041,HB1102,1,Introduced,2023-02-17,UNRWA Accountability and Transparency Act,To withhold United States contributions to the...,2305,House Foreign Affairs,2023-02-17,Referred to the House Committee on Foreign Aff...,https://legiscan.com/US/bill/HB1102/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
1208,1722213,2041,HB1218,1,Introduced,2023-02-27,Expediting Israeli Aerial Refueling Act of 2023,To require the Secretary of the Air Force to p...,2299,House Armed Services,2023-02-27,Referred to the House Committee on Armed Servi...,https://legiscan.com/US/bill/HB1218/2023,https://www.congress.gov/bill/118th-congress/h...,5.0


In [22]:
# Merge votes_df with rollcalls_df to get bill_id into the votes dataset
votes_with_bill_id = pd.merge(votes_df, rollcalls_df[['roll_call_id', 'bill_id']], on='roll_call_id', how='left')

# Select only the relevant columns: 'bill_id', 'roll_call_id', 'people_id', and 'vote'
votes_final = votes_with_bill_id[['bill_id', 'roll_call_id', 'people_id', 'vote_desc']]

# Display the first few rows of the final DataFrame
votes_final.head()

,bill_id,roll_call_id,people_id,vote_desc
0,1650565,1231562,8955,Yea
1,1650565,1231562,8957,Yea
2,1650565,1231562,8963,Yea
3,1650565,1231562,8964,Yea
4,1650565,1231562,8965,Yea


In [25]:
relevant_bills['Score'].value_counts()

Score
5.0    107
4.0     35
3.0     10
1.0      8
Name: count, dtype: int64

In [27]:
##Exploring how many bills were actually voted in Congress

list_of_voted_bills = votes_final['bill_id'].unique().tolist()
list_of_relevant_bills = relevant_bills['bill_id'].unique().tolist()

inBoth = []
counter = 0
for bill in list_of_voted_bills:
    if bill in list_of_relevant_bills:
        counter+=1
        inBoth.append(bill)

print(counter)
print(inBoth)

24
[1657930, 1758028, 1774608, 1780117, 1780826, 1781225, 1782419, 1782534, 1782880, 1784703, 1784708, 1791116, 1792357, 1817842, 1831374, 1838167, 1871712, 1872288, 1872719, 1873083, 1873554, 1873682, 1878308, 1878926]


In [38]:
##Filtering everything to only include the bills that were VOTED ON 
filtered_df = relevant_bills[relevant_bills['bill_id'].isin(inBoth)]

filtered_df.head()

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link,Score
330,1657930,2041,HB340,2,Engrossed,2023-11-02,Hamas and Other Palestinian Terrorist Groups I...,To impose sanctions with respect to foreign su...,0,NaN,2023-11-06,Read the second time. Placed on Senate Legisla...,https://legiscan.com/US/bill/HB340/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
6116,1782419,2041,HB6126,2,Engrossed,2023-11-06,Israel Security Supplemental Appropriations Ac...,Making emergency supplemental appropriations t...,0,NaN,2023-11-14,Cloture motion on the motion to proceed to the...,https://legiscan.com/US/bill/HB6126/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
6669,1791116,2041,HB6679,2,Engrossed,2024-02-01,No Immigration Benefits for Hamas Terrorists Act,To amend the Immigration and Nationality Act w...,2330,Senate Judiciary,2024-02-01,Received in the Senate and Read twice and refe...,https://legiscan.com/US/bill/HB6679/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
7207,1838167,2041,HB7217,1,Introduced,2024-02-05,Israel Security Supplemental Appropriations Ac...,Making emergency supplemental appropriations t...,0,NaN,2024-02-06,On motion to suspend the rules and pass the bi...,https://legiscan.com/US/bill/HB7217/2023,https://www.congress.gov/bill/118th-congress/h...,5.0
8024,1873554,2041,HB8034,1,Introduced,2024-04-17,Israel Security Supplemental Appropriations Ac...,Making emergency supplemental appropriations t...,0,NaN,2024-04-23,"Pursuant to the provisions of H. Res. 1160, H....",https://legiscan.com/US/bill/HB8034/2023,https://www.congress.gov/bill/118th-congress/h...,5.0


In [40]:
#Save the bill_id and their corresponding Score for easy retreival for the next step 
bill_score_dict = filtered_df.set_index('bill_id')['Score'].to_dict()

# Display the dictionary
print(bill_score_dict)
print(len(bill_score_dict))


{1657930: 5.0, 1782419: 5.0, 1791116: 5.0, 1838167: 5.0, 1873554: 5.0, 1878308: 5.0, 1758028: 5.0, 1780117: 5.0, 1780826: 5.0, 1782534: 4.0, 1782880: 5.0, 1784703: 5.0, 1784708: 5.0, 1817842: 5.0, 1831374: 4.0, 1871712: 4.0, 1872288: 4.0, 1872719: 5.0, 1873083: 5.0, 1873682: 4.0, 1878926: 4.0, 1774608: 5.0, 1781225: 5.0, 1792357: 1.0}
24


In [42]:
##Adding Score into teh votes dataframe
filtered_votes_df = votes_final[votes_final['bill_id'].isin(inBoth)]

filtered_votes_score_df = votes_final[votes_final['bill_id'].isin(inBoth)].copy()

# Initialize the Score column with 'None'
filtered_votes_score_df['Score'] = 'None'

# Update the Score column using .loc
for index, row in filtered_votes_score_df.iterrows():
    filtered_votes_score_df.loc[index, 'Score'] = bill_score_dict.get(row['bill_id'], 'None')

# Display the filtered DataFrame
print(filtered_votes_score_df)


        bill_id  roll_call_id  people_id vote_desc Score
46493   1657930       1356906       8955       Nay   5.0
46494   1657930       1356906       8957       Yea   5.0
46495   1657930       1356906       8963       Yea   5.0
46496   1657930       1356906       8964       Yea   5.0
46497   1657930       1356906       8965       Yea   5.0
...         ...           ...        ...       ...   ...
428395  1878926       1443647      24069       Yea   4.0
428396  1878926       1443647      24913       Nay   4.0
428397  1878926       1443647      24963       Nay   4.0
428398  1878926       1443647      24984       Yea   4.0
428399  1878926       1443647      25201       Nay   4.0

[13251 rows x 5 columns]


In [43]:
#Defining the dictionaries that will be used to score each vote for each person. If a person voted yes on a pro israel bill, they will receive a high reward
#If they vote no on an anti Israel Bill they will get a high reward as well and vice versa
ind_scores_yes = {
    5 : 1.0,
    4: 0.8,
    1: 0.2
}

ind_scores_no = {
    5 : 0.2,
    4: 0.4,
    1: 1
}


filtered_votes_score_df['vote_score'] = 'None'  # Initialize the vote_score column with 'None'

##adding in the score for every vote 
for index, row in filtered_votes_score_df.iterrows():
    # Check the vote description
    if row['vote_desc'] == 'Yea':
        filtered_votes_score_df.loc[index, 'vote_score'] = ind_scores_yes.get(row['Score'], 'None')
    elif row['vote_desc'] == 'Nay':
        filtered_votes_score_df.loc[index, 'vote_score'] = ind_scores_no.get(row['Score'], 'None')
    else:
        filtered_votes_score_df.loc[index, 'vote_score'] = 'None'

# Display the DataFrame
print(filtered_votes_score_df)


        bill_id  roll_call_id  people_id vote_desc Score vote_score
46493   1657930       1356906       8955       Nay   5.0        0.2
46494   1657930       1356906       8957       Yea   5.0        1.0
46495   1657930       1356906       8963       Yea   5.0        1.0
46496   1657930       1356906       8964       Yea   5.0        1.0
46497   1657930       1356906       8965       Yea   5.0        1.0
...         ...           ...        ...       ...   ...        ...
428395  1878926       1443647      24069       Yea   4.0        0.8
428396  1878926       1443647      24913       Nay   4.0        0.4
428397  1878926       1443647      24963       Nay   4.0        0.4
428398  1878926       1443647      24984       Yea   4.0        0.8
428399  1878926       1443647      25201       Nay   4.0        0.4

[13251 rows x 6 columns]


In [45]:
# Ensure the 'vote_score' column is numeric
filtered_votes_score_df['vote_score'] = pd.to_numeric(filtered_votes_score_df['vote_score'], errors='coerce')

# Now, group by 'people_id' and sum the 'vote_score' for each group
people_vote_scores = filtered_votes_score_df.groupby('people_id')['vote_score'].sum().reset_index()

# Optionally, rename the columns for clarity
people_vote_scores.columns = ['people_id', 'total_vote_score']

# Display the resulting DataFrame
print(people_vote_scores)


     people_id  total_vote_score
0         8955              14.8
1         8957              20.6
2         8963              19.8
3         8964              19.0
4         8965              20.6
..         ...               ...
534      24913              19.0
535      24958               2.0
536      24963              12.8
537      24984              17.4
538      25201               2.0

[539 rows x 2 columns]


In [47]:
# Initialize the new columns
people_vote_scores['Name'] = 'None'
people_vote_scores['Role'] = 'None'
people_vote_scores['District'] = 'None'
people_vote_scores['Party'] = 'None'

# Create a dictionary for quick lookup from people_df
people_lookup = people_df.set_index('people_id').to_dict(orient='index')

# Iterate through people_vote_scores and populate the columns
for index, row in people_vote_scores.iterrows():
    person_id = row['people_id']
    
    if person_id in people_lookup:
        person_info = people_lookup[person_id]
        people_vote_scores.at[index, 'Name'] = person_info['name']
        people_vote_scores.at[index, 'Role'] = person_info['role']
        people_vote_scores.at[index, 'District'] = person_info['district']
        people_vote_scores.at[index, 'Party'] = person_info['party_id']

# Display the updated DataFrame
print(people_vote_scores)

     people_id  total_vote_score                Name Role  District Party
0         8955              14.8     Nydia Velazquez  Rep   HD-NY-7     1
1         8957              20.6       Frank Pallone  Rep   HD-NJ-6     1
2         8963              19.8        Kathy Castor  Rep  HD-FL-14     1
3         8964              19.0     Gerald Connolly  Rep  HD-VA-11     1
4         8965              20.6       Henry Cuellar  Rep  HD-TX-28     1
..         ...               ...                 ...  ...       ...   ...
534      24913              19.0  Jennifer McClellan  Rep   HD-VA-4     1
535      24958               2.0     Laphonza Butler  Sen     SD-CA     1
536      24963              12.8            Gabe Amo  Rep   HD-RI-1     1
537      24984              17.4       Celeste Maloy  Rep   HD-UT-2     2
538      25201               2.0     Timothy Kennedy  Rep  HD-NY-26     1

[539 rows x 6 columns]


In [72]:
import plotly.graph_objects as go

# Map party_id to colors
party_colors = {1: 'blue', 2: 'red'}

# Create scatter plots for each party
fig = go.Figure()

# Add trace for Party 1 (Democrats, Blue)
fig.add_trace(go.Scatter(
    x=people_vote_scores[people_vote_scores['Party'] == 1]['total_vote_score'],
    y=people_vote_scores[people_vote_scores['Party'] == 1]['people_id'],
    mode='markers',
    marker=dict(color='blue'),
    name='Party (D)',  # Legend entry for Democrats
    hovertext=people_vote_scores[people_vote_scores['Party'] == 1]['Name'],
    hoverinfo='text+x+y'
))

# Add trace for Party 2 (Republicans, Red)
fig.add_trace(go.Scatter(
    x=people_vote_scores[people_vote_scores['Party'] == 2]['total_vote_score'],
    y=people_vote_scores[people_vote_scores['Party'] == 2]['people_id'],
    mode='markers',
    marker=dict(color='red'),
    name='Party (R)',  # Legend entry for Republicans
    hovertext=people_vote_scores[people_vote_scores['Party'] == 2]['Name'],
    hoverinfo='text+x+y'
))

# Add background color for the x-axis range 0-3
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=0,
    x1=3,
    y0=0,
    y1=1,
    fillcolor="gray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add buttons for filtering
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"visible": [True, True]}],
                    label="All",
                    method="update"
                ),
                dict(
                    args=[{"visible": [True, False]}],
                    label="Democrats",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False, True]}],
                    label="Republicans",
                    method="update"
                ),
            ]),
        )
    ],
    title="Total Vote Score by People",
    xaxis_title="Total Vote Score",
    yaxis_title="People ID",
    showlegend=True,
    xaxis=dict(
        showticklabels=False  # Hides the x-axis tick labels
    ),
    annotations=[
        dict(
            x=0, y=-0.1,  # Position for "Anti Israel"
            xref='paper', yref='paper',
            text='Anti Israel',
            showarrow=False,
            xanchor='left'
        ),
        dict(
            x=1, y=-0.1,  # Position for "Pro Israel"
            xref='paper', yref='paper',
            text='Pro Israel',
            showarrow=False,
            xanchor='right'
        )
    ]
)

fig.add_annotation(
    x=1.5,  # Position the text in the middle of the shaded region (between 0 and 3)
    y=1.1,  # Position it slightly above the plot area
    xref="x",
    yref="paper",
    text="Senators",  # The text to display
    showarrow=False,  # No arrow, just text
    font=dict(size=14, color="black"),  # Customize the font size and color
    xanchor="center"
)


# Update the legend and marker properties
fig.update_traces(
    marker=dict(size=15, opacity=0.6),
)

fig.show()


In [59]:
#Final Sorted Dataframe of all the CongressMen and their "Pro Israelity"
sortedPeople = people_vote_scores.sort_values(by='total_vote_score', ascending=False)

print(sortedPeople)


     people_id  total_vote_score               Name Role  District Party
385      21646              27.0        Mike Garcia  Rep  HD-CA-27     2
269      18287              26.2          Don Bacon  Rep   HD-NE-2     2
455      23182              26.2  Rudolph Yakym III  Rep   HD-IN-2     2
471      24012              26.2  Monica De La Cruz  Rep  HD-TX-15     2
154      11077              26.2        Tim Walberg  Rep   HD-MI-5     2
..         ...               ...                ...  ...       ...   ...
137       9484               1.2     Chuck Grassley  Sen     SD-IA     2
75        9210               1.2       John Boozman  Sen     SD-AR     2
168      11807               1.0    David Cicilline  Rep   HD-RI-1     1
153      11074               0.2          Tim Scott  Sen     SD-SC     2
195      14918               0.0      Chris Stewart  Rep   HD-UT-2     2

[539 rows x 6 columns]


In [62]:
#Splitting the Senators and the Represantatives

senators_df = people_vote_scores[people_vote_scores['Role'] == 'Sen'].sort_values(by='total_vote_score', ascending=False)

reps_df = people_vote_scores[people_vote_scores['Role'] == 'Rep'].sort_values(by='total_vote_score', ascending=False)

print("\nRepresentatives DataFrame:")
print(reps_df)
print("Senators DataFrame:")
print(senators_df)





Representatives DataFrame:
     people_id  total_vote_score              Name Role  District Party
385      21646              27.0       Mike Garcia  Rep  HD-CA-27     2
496      24037              26.2       Nick LaLota  Rep   HD-NY-1     2
156      11099              26.2      Steve Womack  Rep   HD-AR-3     2
424      21964              26.2       Cliff Bentz  Rep   HD-OR-2     2
167      11606              26.2  David Schweikert  Rep   HD-AZ-1     2
..         ...               ...               ...  ...       ...   ...
538      25201               2.0   Timothy Kennedy  Rep  HD-NY-26     1
192      14903               1.4  Markwayne Mullin  Rep   HD-OK-2     2
275      18294               1.4          Ted Budd  Rep  HD-NC-13     2
168      11807               1.0   David Cicilline  Rep   HD-RI-1     1
195      14918               0.0     Chris Stewart  Rep   HD-UT-2     2

[442 rows x 6 columns]
Senators DataFrame:
     people_id  total_vote_score              Name Role District